In [3]:
##NOTEBOOK FOR PLOTLY DASH LAB. CODE WAS RUN IN THE SKILL NETWORK LABS ENVIRONMENT.

# Import required libraries
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
dcc.Dropdown(id='site-dropdown',
                                            options=[
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                            {'label': 'All Sites', 'value': 'ALL'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                    min=0, max=10000, step=1000,
                                                    marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                                    value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(site_dropdown):
    filtered_df=spacex_df
    if site_dropdown == 'ALL':
        fig = px.pie(filtered_df, values='class',  names='Launch Site', title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        filtered_LC = filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        title_pie = f"Total Success Launches for site {site_dropdown}"     
        fig = px.pie(filtered_LC, values='class count', names='class', title=title_pie)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property="value")])

def scatter_chart(site, payload_slider):
    low, high=payload_slider
    cond=(spacex_df['Payload Mass (kg)']>low) & (spacex_df['Payload Mass (kg)']<high)
    filtered_df=spacex_df[cond]
    if site =='ALL':
        fig2=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between payload and success for all sites')
        return fig2
    else:
        fig2=px.scatter(filtered_df[filtered_df['Launch Site']==site], x='Payload Mass (kg)', y='class', color='Booster Version Category', 
        title='Correlation between payload and success for {site}')
        return fig2

# Run the app
if __name__ == '__main__':
    app.run_server()

FileNotFoundError: [Errno 2] No such file or directory: 'spacex_launch_dash.csv'